# 验证和导入项目元数据 <a class="anchor" id="top"></a>

在这个Jupyter笔记本中，您将从 `01_Validating_and_Importing_User_Item_Interaction_Data.ipynb` 中结束的地方开始来构建一个项目元数据集。这将允许您使用过滤器，以及 `User-Personalization` 或者 `HRNN-Metadata` 算法。

要运行当前这个笔记本，您需要运行之前的笔记本 `01_Validating_and_Importing_User_Item_Interaction_Data`，其中您创建了一个数据集然后将交互数据导入到 Amazon Personalize 中，并且在之前的笔记本最后保存了一些变量，现在您需要将这些值加载到这个笔记本中。

In [ ]:
%store -r

## 准备项目元数据 <a class="anchor" id="prepare"></a>
[回到顶部](#top)

接下来要做的事情是加载数据并确认数据处于一个良好状态，然后将其保存到CSV中，以便 Amazon Personalize 使用。

首先，导入数据科学常用的Python库集合。

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

接下来，打开数据文件并查看前几行数据。

In [ ]:
original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data.head(5)

In [ ]:
original_data.describe()

这其实没有告诉我们关于数据集的更多信息，所以我们将探索更多的原始信息。可以看到类型 genres 通常被组合在一起放到列表中，这对我们来说很好，因为 Personalize 支持这种结构。

In [ ]:
original_data.info()

由此，您可以看到数据集中共有3列条目（完整数据集为62,000+，小数据集为9742）。

这是一个非常小的数据集，只有电影ID movieId、电影名称 title 和适用于每个条目的类型列表 genres。然而，在Movielens数据集中还有额外的数据可用，例如，片名中包括了电影发行的年份。让我们再创建一列元数据。

In [ ]:
original_data['year'] =original_data['title'].str.extract('.*\((.*)\).*',expand = False)
original_data.head(5)

从项目元数据的角度来看，我们只希望包含与训练模型和/或过滤结果相关的信息，因此我们将删除名称列，保留类型信息。

In [ ]:
itemmetadata_df = original_data.copy()
itemmetadata_df = itemmetadata_df[['movieId', 'genres', 'year']]
itemmetadata_df.head()

在数据处理之后，务必要确认数据格式是否已更改。

In [ ]:
itemmetadata_df.dtypes

Amazon Personalize有一个默认列 `ITEM_ID`，它将映射到我们的 `movieId`，现在可以通过指定 `GENRE` 来充实信息。

In [ ]:
itemmetadata_df.rename(columns = {'genres':'GENRE', 'movieId':'ITEM_ID', 'year':'YEAR'}, inplace = True) 

现在数据已经准备好啦！我们只需要将其保存为CSV文件。

In [ ]:
itemmetadata_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+itemmetadata_filename), index=False, float_format='%.0f')

In [ ]:
# 配置 Boto3 的 Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### 创建数据集

首先，定义一个模式来告诉 Amazon Personalize 要上传的数据集类型。根据数据集的类型，模式中有几个保留和需要强制使用的关键字。更详细的信息可以在[文档](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)中找到。

在这里，您将为项目元数据创建一个Schema，它需要 `ITEM_ID` 和 `GENRE` 字段。它们在Schema中定义的顺序必须与它们在数据集中出现的顺序相同。

In [ ]:
itemmetadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        },{
            "name": "YEAR",
            "type": "int",
        },
        
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-item",
    schema = json.dumps(itemmetadata_schema)
)

itemmetadataschema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

创建Schema后，您可以在数据集组中创建数据集。注意，这时还没有加载数据，我们将在稍后几步操作。

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = itemmetadataschema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### 上传数据到 S3

现在已经创建了 Amazon S3 桶，接下来上传用户-项目交互数据的CSV文件。

In [ ]:
itemmetadata_file_path = data_dir + "/" + itemmetadata_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

## 导入项目元数据 <a class="anchor" id="import"></a>
[回到顶部](#top)

前面您创建了数据集组和数据集来存放信息，现在执行导入作业，将数据从S3桶加载到 Amazon Personalize 数据集。

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-item-import1",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

数据集之前，导入作业必须是活动的。执行下面的单元格并等待它显示活动状态，它会每60秒钟检查一次导入作业的状态，最多等待6个小时。

导入数据可能需要一些时间，这取决于数据集的大小。在这个实验中，数据导入作业大约需要15分钟。

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

导入完成后，您就可以启用推荐过滤功能以及 `HRNN-Metadata`。运行下面的单元格，然后继续存储一些值以便在下一个笔记本中使用。完成单元格后，打开笔记本`03_Creating_and_Evaluating_Solutions.ipynb`继续。

In [ ]:
%store items_dataset_arn
%store itemmetadataschema_arn